# 字典数据结构（dictionary）

### Python内置的dict是使用哈希表实现的。

字典数据结构（dict_adt.py）

主要是几个专门的方法，基本的元素操作方法已经在哈希表中实现。

注意items等方法在python3中返回的是迭代器。

### Hashable
作为 dict 的 key 必须是可哈希的，也就是说不能是 list 等可变对象。

>An object is hashable if it has a hash value which never changes during its lifetime (it needs a hash() method), and can be compared to other objects (it needs an eq() or cmp() method). Hashable objects which compare equal must have the same hash value.

>Hashability makes an object usable as a dictionary key and a set member, because these data structures use the hash value internally.

>All of Python’s immutable built-in objects are hashable, while no mutable containers (such as lists or dictionaries) are. Objects which are instances of user-defined classes are hashable by default; they all compare unequal (except with themselves), and their hash value is derived from their id().

In [3]:
class Array(object):
    """继承：object
    方法：初始化、取值（下标访问）、设置值（安下标设置）、取长度、清空、迭代
    """
    def __init__(self, size=32, init=None):  # 加入每个槽位的初始值，还是默认为None
        self._size = size
        self._items = [init] * size

    def __getitem__(self, index):  # Called to implement evaluation of self[index]实现下标访问.
        return self._items[index]

    def __setitem__(self, index, value):  # Called to implement assignment to self[index].
        self._items[index] = value

    def __len__(self):
        return self._size

    def clear(self, value=None):
        for i in range(len(self._items)):
            self._items[i] = value

    def __iter__(self):
        for item in self._items:
            yield item


class Slot(object):
    """定义一个 hash 表 数组的槽
        注意，一个槽有三种状态，看你能否想明白。相比链接法解决冲突，二次探查法删除一个 key 的操作稍微复杂。
        1.从未使用过，值为HashMap.UNUSED（None）。此槽没有被使用和冲突过，查找时只要找到 UNUSED 就不用再继续探查了
        2.使用过但是 remove 了，此时值为 HashMap.EMPTY，该探查点后边的元素仍可能是有key
        3.槽正在使用 Slot 节点
    """

    def __init__(self, key, value):
        '''初始化键值'''
        self.key, self.value = key, value


class HashTable(object):
    """key-value对以slot对象的形式保存在数组中
    slot初始值为HashMap.UNUSED
    """
    UNUSED = None
    EMPTY = Slot(None, None)

    def __init__(self):
        # _table是一个Array对象，默认大小为32，初始时候为空
        self._table = Array(8, init=HashTable.UNUSED)  # key-value对以slot对象的形式保存在数组中，slot初始值为HashMap.UNUSED
        self.length = 0

    @property
    def _load_factor(self):
        # 负载因子超过0.8重新分配空间
        return self.length / float(len(self._table))

    def __len__(self):
        '''长度'''
        return self.length

    def _hash(self, key):
        '''求hash值，返回结果的绝对值'''
        return abs(hash(key)) % len(self._table)

    def _find_key(self, key):
        """查找key，返回key在数组中的位置
            若index位置的值为UNUSED，说明槽未使用过，key在数组中不存在
            若为EMPTY或key值，则设法返回key在数组中的位置
        """
        index = self._hash(key)
        _len = len(self._table)
        while self._table[index] is not HashTable.UNUSED:
            if self._table[index] is HashTable.EMPTY:  # 若值为EMPTY，继续探查点后边的元素
                index = (index * 5 + 1) % _len
                continue  # 跳过当前循环的剩余语句
            elif self._table[index].key == key:  # 若值不为EMPTY，检查slot的key值是否等于key
                return index
            else:  # 有值且不为key，则继续探查点后边的元素
                index = (index * 5 + 1) % _len
        return None

    def _find_slot_for_insert(self, key):
        '''发现对应的slot进行值的插入'''
        index = self._hash(key)
        _len = len(self._table)
        while not self._slot_can_insert(index):
            index = (index * 5 + 1) % _len
        return index

    def _slot_can_insert(self, index):
        '''当HashTable的标记为Empty或者是UNUSED的时候能够插入'''
        return self._table[index] is HashTable.EMPTY or self._table[index] is HashTable.UNUSED

    def __contains__(self, key):  
        # 实现散列表的 in 操作
        index = self._find_key(key)
        return index is not None

    def add(self, key, value):
        """向散列表中添加key-value对
        首先查找key是否已经在散列表中，若已存在，重置其value并返回False；
        若不存在，查找可供插入的槽并插入Slot，并检查负载因子，若有必要则进行重哈希
        """
        index = self._find_key(key) # 查找key值是否已经在散列表中
        if index is not None:  # 更新已存在的key
            self._table[index].value = value
            return False
        else: # 不在散列表中可以寻找槽进行插入
            index = self._find_slot_for_insert(key)
            self._table[index] = Slot(key, value)  # 用key-value对构造Slot并插入到数组
            self.length += 1
            if self._load_factor >= 0.8:
                self._rehash()
            return True

    def _rehash(self):
        '''重新hash
        1、使用新的大小构造数组
        2、迭代旧表将非空节点添加到数组
        '''
        oldtable = self._table
        newsize = len(self._table) * 2
        self._table = Array(newsize, HashTable.UNUSED)  # 使用新的大小构造数组
        self.length = 0

        for slot in oldtable:  # 迭代旧表并将非空节点添加到新数组
            if slot is not HashTable.UNUSED and slot is not HashTable.EMPTY:
                index = self._find_slot_for_insert(slot.key)
                self._table[index] = slot
                self.length += 1

    def get(self, key, default=None):
        '''根据key值查找返回对应的value'''
        index = self._find_key(key)
        if index is None:  # 首先检查key是否存在
            return default
        else:
            return self._table[index].value

    def remove(self, key):
        '''根据key值去除对应的value'''
        index = self._find_key(key)
        if index is None:  # 首先检查key是否存在
            raise KeyError()
        else:
            value = self._table[index].value
            self.length -= 1
            self._table[index] = HashTable.EMPTY
            return value

    def __iter__(self):
        """迭代数组中的slot，若slot不为空，则返回其key值
        :return:
        """
        for slot in self._table:
            if slot not in (HashTable.EMPTY, HashTable.UNUSED):
                yield slot.key


class DictADT(HashTable):
    """字典类型的数据和HASH表的大同小异
    设置值、获取值、按照插槽迭代、出字典、返回值、获取键、获取值
    """

    def __setitem__(self, key, value):
        return self.add(key, value)

    def __getitem__(self, key):
        if key not in self:
            raise KeyError()
        else:
            return self.get(key)

    def _iter_slot(self):
        '''迭代_table对应的Array，不为EMPTY或者UNUSED的时候就返回slot（其实就是一个项item）'''
        for slot in self._table:
            if slot not in (HashTable.EMPTY, HashTable.UNUSED):
                yield slot

    def pop(self, key, default):
        '''弹出元素并去除'''
        if default and key not in self:
            return default
        else:
            return self.remove(key)

    def items(self):
        '''返回对应的item，调用_iter_slot，返回的就是key—value对'''
        for slot in self._iter_slot():
            yield (slot.key, slot.value)

    def keys(self):
        '''返回key'''
        for slot in self._iter_slot():
            yield slot.key

    def values(self):
        '''返回value'''
        for slot in self._iter_slot():
            yield slot.value


def test_dict_adt():
    # 测试字典
    import random
    # 实例化一个字典
    d = DictADT()

    d['a'] = 1
    assert d['a'] == 1
    d.remove('a')

    # 随机定义一个打乱循序的列表并将值保存到列表里面
    ll = list(range(30))
    random.shuffle(ll)
    for i in ll:
        d.add(i, i)

    for i in range(30):
        assert d.get(i) == i
    # 字典的keys排序顺序和定义的列表的顺序相等
    assert sorted(list(d.keys())) == sorted(ll)
    assert sorted(list(d)) == sorted(ll)

test_dict_adt()